In [24]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 200)

import warnings
warnings.filterwarnings("ignore")

In [25]:
from bs4 import BeautifulSoup
import requests

import missingno as ms
import time
import random
import datetime
from datetime import timedelta

In [26]:
import datefinder
# Example
string_with_dates = "“Family Feud” 11/23/22"
matches = datefinder.find_dates(string_with_dates)
next(matches);

In [27]:
import datetime
from datetime import timedelta
from datetime import datetime

# bobbymgsk

In [242]:
def parse_bobby_to_soup():
    all_pages = ""
    for page in range(1,150):
        url = f'https://bobbymgsk.wordpress.com/category/family-feud/page/{page}/'
        try:
            time.sleep(5*random.random())
            page = requests.get(url)
            all_pages += page.text
        except:
            pass
        
    
    soup = BeautifulSoup(all_pages, 'html.parser')
    
    return soup

In [ ]:
soup = parse_bobby_to_soup()

## Posts

In [235]:
# Get list of posts (most posts contain two episodes)
# Number of posts is 1260 as of 12/29/2022
posts = soup.find_all("div", {"class": "post"})
len(posts)

1

In [140]:
def get_post_version(title):
    if title:
        return "Celebrity" if "Celebrity" in title else "Standard"
    return None

In [179]:
posts_list = []
for post in posts:
#     postTitle = post.find("h3").text
#     postText = post.get_text()
#     postHTML = post
#     postVersion = get_post_version(postTitle)
#     posts_list.append([postTitle, postHTML, postText, postVersion])
    a_tags = post.find_all('a', href=True)
    for a in a_tags:
        href = a['href']
        if 'category' not in href and 'xmlrpc' not in href:
            posts_list.append(href)
            break
    
posts_list.reverse()

In [181]:
len(posts_list)

1272

In [182]:
posts = pd.DataFrame(posts_list, columns=['PostURL'])
posts['PostID'] = posts.index+1
posts = posts[['PostID','PostURL']]
posts = posts.drop_duplicates()

In [180]:
posts_list[0:10]

['https://bobbymgsk.wordpress.com/2016/01/19/family-feud-118/',
 'https://bobbymgsk.wordpress.com/2016/01/20/family-feud-119/',
 'https://bobbymgsk.wordpress.com/2016/01/21/family-feud-12016/',
 'https://bobbymgsk.wordpress.com/2016/01/21/family-feud-121/',
 'https://bobbymgsk.wordpress.com/2016/01/26/family-feud-12516/',
 'https://bobbymgsk.wordpress.com/2016/01/27/family-feud-12616/',
 'https://bobbymgsk.wordpress.com/2016/01/28/family-feud-12716/',
 'https://bobbymgsk.wordpress.com/2016/01/29/family-feud-12816/',
 'https://bobbymgsk.wordpress.com/2016/01/29/family-feud-12916/',
 'https://bobbymgsk.wordpress.com/2016/02/02/family-feud-2116/']

In [ ]:
# posts = pd.DataFrame(posts_list, columns=['PostTitle','PostHTML','PostText','PostVersion'])
# posts['PostID'] = posts.index+1
# posts = posts[['PostID','PostTitle','PostHTML','PostText','PostVersion']]
# posts = posts.drop_duplicates()

In [183]:
posts.head()

,PostID,PostURL
0,1,https://bobbymgsk.wordpress.com/2016/01/19/fam...
1,2,https://bobbymgsk.wordpress.com/2016/01/20/fam...
2,3,https://bobbymgsk.wordpress.com/2016/01/21/fam...
3,4,https://bobbymgsk.wordpress.com/2016/01/21/fam...
4,5,https://bobbymgsk.wordpress.com/2016/01/26/fam...


In [145]:
posts_ = pd.merge(posts, b24[['PostTitle','PostDate']].drop_duplicates(), left_on='PostTitle', right_on='PostTitle', how="left")
posts_ = posts_.drop_duplicates()

In [146]:
posts_ = posts_[['PostID','PostTitle','PostDate','PostHTML','PostText','PostVersion']]
posts_.columns = ['PostID','PostTitle','PostDate','PostHTML','PostText','PostVersion']

In [184]:
posts.to_csv('post_urls.csv', index=False)

In [148]:
len(posts_)

1272

## Post Comments

In [216]:
fct_post = pd.read_csv('fct_post.csv')

In [ ]:
comment_list = []
for index, row in fct_post.iterrows():
    time.sleep(3*random.random())
    url = row['PostURL']
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    comment_author = [author.text.replace('\n','').replace(' said:','') for author in soup.find_all("div", {"class": "comment-author"})]
    comment_text = [comment.find('p').text for comment in soup.find_all("div", {"class": "commentText"})]
    comment_date = [comment.find('a').text for comment in soup.find_all("div", {"class": "commentDate"})]
    
    
    for i in range(len(comment_author)):
        comment_list.append([row['PostID'], i+1, comment_author[i], comment_text[i], comment_date[i]])

In [236]:
comms = pd.DataFrame(comment_list, columns = ['PostID','CommentOrder','Author','Comment','CommentDate'])
comms['CommentID'] = comms.index+1
comms = comms[['CommentID','PostID','CommentOrder','Author','Comment','CommentDate']]

In [241]:
fct_comment = comms[['CommentID','PostID','CommentOrder','Author','Comment']]
fct_comment_date = comms[['CommentID','CommentDate']]

fct_comment.to_csv('fct_comment.csv', index=False)
fct_comment_date.to_csv('fct_comment_date.csv', index=False)

## Games

In [ ]:
# Games can start with random titles ...yay... 
# the following list of titles each designate a new game
with open("game_titles.txt", "r", encoding="UTF-8") as f:
    game_titles = [x.strip().replace('"', '') for x in f.readlines()]
    
game_titles[0:10];

### Display post that have more fast money rounds than game titles
There is a fast money in every game, so this is effectively checking if the number of game titles matches the number of games. For example, some rematchs don't have a game title, those are some of what appears below

In [ ]:
for post in posts: 
    # Keep track of the number of fast money's per post
    # Since every game has a fast money (and they are all denoted with "FM:"), we can use this to
    # track which games are missing crucial information, like a title or family list
    fastmoney_count = 0
    
    # keep track of each game
    games = []
    
    games.append(post.find('a', href=True).text)

    p_tags = post.find_all('p')
    for p_tag in p_tags:
        if 'FM:' in p_tag.text:
            fastmoney_count += 1
        if bool([ele for ele in game_titles if(ele in p_tag.text)]):
            games.append(p_tag.text)
            
    if len(games) < fastmoney_count+1:
        for d in games: print(d)

### CREATE A DATAFRAME OF FAMILY FEUD GAMES, HALLELUJAH!!

In [ ]:
game_count = 0
all_games = []

for post in posts:
    
    post_title = post.find('a', href=True).text
    game_text = ""
    game_html = ""
    
    def new_game(p_tag):
        return bool([ele for ele in game_titles if(ele in p_tag.text)])
    
    for p_tag in post.find_all('p'):
        if new_game(p_tag):
            if len(game_text) > 0:
                all_games.append([post_title, game_text, game_html])
            game_text = p_tag.text
            game_html = str(p_tag)
        else:
            if len(p_tag.text) > 0 : 
                game_text += ' || ' + p_tag.text
                game_html += ' || ' + str(p_tag)
    if len(game_text) > 0:
        all_games.append([post_title, game_text, game_html])

In [ ]:
df = pd.DataFrame(all_games, columns = ['PostTitle','GameText','GameHTML'])

df = df.replace("Lewises v. Days", "Miles v. Days", regex=True)

In [ ]:
def get_post_date(post_title):
    try:
        dates = datefinder.find_dates(post_title)
        return next(dates)
    except:
        return ""

def get_post_version(title):
    if "Celebrity" in title:
        return "Celebrity"
    else:
        return "Standard"
    
def get_game_title(game_text):
    for p_text in game_text.split('||'):
        if bool([ele for ele in game_titles if(ele in p_text)]):
            game_title = p_text
            return game_title
        
def get_family_info(title, fam_num):
    try:
        title = title.split(':')[1]
        family = re.split(' v. | v | vs. | vs ', title)[fam_num]
    except:
        family = ""
        
    for g in game_titles:
        family = family.replace(g,'').replace(':','').strip()
    family = family.replace('(','$$@(').split('$$@')
    name = family[0]
#     info = family[1:]
    return name

In [ ]:
df['PostVersion'] = df['PostTitle'].apply(get_post_version)
df['PostDate'] = df['PostTitle'].apply(get_post_date)
df['GameTitle'] = df['GameText'].apply(get_game_title)
df['FamilyA_p'] = df['GameTitle'].apply(lambda x: get_family_info(x, 0))
df['FamilyB_p'] = df['GameTitle'].apply(lambda x: get_family_info(x, 1))

In [ ]:
df = (df.fillna("")
    .apply(lambda x: x.str.strip() if x.dtype == "object" else x)  
    .replace("’", "'", regex=True))

### Replace typos from Bobby
Determined by comparison from Trakt titles, mostly confirmed via video clips. Why does he do plural last names? Agh!

In [ ]:
def replace_typos_bobbymgsk(df):
    replacements = (('Crovetttos','Crovettos'),
        ('Collingsworth','Hollingsworth'),
        ('Aladenoyers','Aladenoyes'),
        ('McLards','McClards'),
        ('Olszweski','Olszewski'),
        ('McLards','McClards'),
        ('Hlaseses','Hlases'),
        ('Cartotenutos','Carotenutos'),
        ('Boulhac','Roulhac'),
        ('Coanses','Coans'),
        ('Chesnut','Chestnut'),
        ('Keningsbergs','Kenigsberg'),
        ('Mokiaoses','Mokiaos'),
        ('Iaintellos','Ianitellos')
    )

    for r in replacements:
        df = df.replace(r[0],r[1],regex=True)
        
    return df

df = replace_typos_bobbymgsk(df)

### De-pluralize family names

In [ ]:
def depluralize_family(name):
    df_trakt = pd.read_pickle("pickles/df_trakt.pkl")
    trakt_names = [t.strip() for t in list(df_trakt.FamilyA) + list(df_trakt.FamilyB) if len(t.strip())>0]
    trakt_names_ending_e = list(set([t for t in trakt_names if t[-1] == 'e']))

    try:
        name = name.strip()
        if name in ['Wis']:
            return name
        if (name[-2:] == 'es') and (name[:-1] in trakt_names_ending_e):
            return name[:-1] 
        if name[-2:] == 'es':
            return name[:-2]
        elif name[-1] == 's':
            return name[:-1]
        else:
            return name
    except:
        return name

df['FamilyA'] = df['FamilyA_p'].apply(depluralize_family)
df['FamilyB'] = df['FamilyB_p'].apply(depluralize_family)

### Overwrites

In [ ]:
game_overwrites = pd.read_csv("game_overwrites.csv", names=['PostTitle','GameTitle','FamilyA','FamilyB','FamilyA_p','FamilyB_p']).fillna("")
game_overwrites = game_overwrites.replace('"F','“F',regex=True).replace('d"','d”',regex=True)

In [ ]:
df['Overwritten'] = 0

In [ ]:
for overwrite_index, overwrite_row in game_overwrites.iterrows():
    games_by_post = df[df['PostTitle'] == overwrite_row['PostTitle']]
    for game_index, game_row in games_by_post.iterrows():
        if overwrite_row['FamilyA'] in game_row['GameText']:
            pairs = [['Overwritten', 1],
                 ['FamilyA',     overwrite_row['FamilyA']],
                 ['FamilyB',     overwrite_row['FamilyB']],
                 ['FamilyA_p',   overwrite_row['FamilyA_p']],
                 ['FamilyB_p',   overwrite_row['FamilyB_p']]]
            for pair in pairs:
                df.loc[(df['PostTitle'] == game_row['PostTitle']) &
                       (df['GameTitle'] == game_row['GameTitle']), pair[0]] = pair[1]

### Pre-made Bobby Assignments

In [ ]:
bobby_assignments = pd.read_csv('bobby_assignments.csv', encoding="windows-1252", names=['SeasonNum','EpisodeNum','FamilyA','FamilyB','PostTitle'])

bobby_assignments['FamilyA'] = bobby_assignments['FamilyA'].str.strip()
bobby_assignments['FamilyB'] = bobby_assignments['FamilyB'].str.strip()

In [ ]:
def match_to_assignment(df_row, typ):
    
    title = df_row['PostTitle']

    a = pd.DataFrame(df_row)
    b = bobby_assignments[bobby_assignments['PostTitle'] == title]

    for ib, row_b in b.iterrows():
        if (row_b['FamilyA'] in df_row['GameText']) and (row_b['FamilyB'] in df_row['GameText']):
            if typ == 'Season':
                return row_b['SeasonNum']
            else:
                return row_b['EpisodeNum']

In [ ]:
df['EpisodeNum'] = df.apply(lambda x: match_to_assignment(x, 'Episode'), axis=1)
df['SeasonNum'] = df.apply(lambda x: match_to_assignment(x, 'Season'), axis=1)

In [ ]:
df.loc[(df['PostDate'] >= '2010-09-01') & (df['PostDate'] <= '2011-08-01'), 'SeasonNum'] = 12
df.loc[(df['PostDate'] >= '2011-09-01') & (df['PostDate'] <= '2012-08-01'), 'SeasonNum'] = 13

In [ ]:
df[df.PostVersion == 'Standard'].to_csv('df.csv')

<span>-------------------------------------------------------------------------------------------------------------------------------------------------------


<span>-------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
benchmark = pd.read_csv('benchmark.csv')

# Forgot suddenn death (SD) smh...
split_string = '(R1|R2|\|\| Double|\|\| DOUBLE|Double:|\|\| Triple|\|\| TRIPLE|TRIPLE:|FM|Unsaid #1)'
question_split = benchmark['GameText'].str.split(split_string, expand=True)
benchmark1 = pd.concat([benchmark, question_split], axis=1)
benchmark1.to_csv('benchmark1.csv', index=False)

In [ ]:
benchmark2 = pd.read_csv('benchmark2.csv')

split_string = '(SD)'
sd_split = benchmark2['8'].str.split(split_string, expand=True)
benchmark3 = pd.concat([benchmark2, sd_split], axis=1)
benchmark3.to_csv('benchmark3.csv', index=False)

In [ ]:
benchmark4 = pd.read_csv('benchmark4.csv')

In [ ]:
split_string = '(#1)'
r1_split = benchmark4['QS_Round1Text'].str.split(split_string, 1, expand=True)
r2_split = benchmark4['QS_Round2Text'].str.split(split_string, 1, expand=True)
r3_split = benchmark4['QS_DoubleText'].str.split(split_string, 1, expand=True)
r4_split = benchmark4['QS_TripleText'].str.split(split_string, 1, expand=True)

benchmark5 = pd.concat([benchmark4, r1_split, r2_split, r3_split, r4_split], axis=1)
benchmark5.to_csv('benchmark5.csv', index=False)

In [ ]:
benchmark6 = pd.read_csv('benchmark6.csv')

def getNumOfAnswers(answerText):
    for i in range(8,-1,-1):
        if f"#{i}" in str(answerText) or f"# {i}" in str(answerText):
            return i
    return 0

benchmark6['QS_Round1A_NumAnswers'] = benchmark6['QS_Round1A'].apply(getNumOfAnswers)
benchmark6['QS_Round2A_NumAnswers'] = benchmark6['QS_Round2A'].apply(getNumOfAnswers)
benchmark6['QS_DoubleA_NumAnswers'] = benchmark6['QS_DoubleA'].apply(getNumOfAnswers)
benchmark6['QS_TripleA_NumAnswers'] = benchmark6['QS_TripleA'].apply(getNumOfAnswers)

benchmark6['QS_TripleA_NumAnswers'].value_counts().sort_index()

benchmark6[benchmark6['QS_TripleA_NumAnswers'] < 4]

In [ ]:
benchmark7 = pd.read_csv('benchmark7.csv')

benchmark7['QS_SuddenDeathFullText'] = (benchmark7['QS_SuddenDeathFullText']
    .str.replace('?','?##')
    .str.replace('.','.##')
    .str.replace('_\n','_\n##'))
# benchmark7['QS_SuddenDeathFullText'][0]
sd_split = benchmark7['QS_SuddenDeathFullText'].str.split('##', 1, expand=True)

sd_split.columns = ['A','B']
sd_split[(sd_split['A'].notnull()) & (sd_split['B'].isnull())]

In [ ]:
benchmark8 = pd.concat([benchmark7, sd_split], axis=1)
benchmark8.to_csv('benchmark8.csv', index=False)

In [ ]:
benchmark9 = pd.read_csv('benchmark9.csv')

split_string = '(\n#1|#1|#2|#3|#4|#5)'
fm_split = benchmark9['QS_FastMoneyFullText'].str.split(split_string, 5, expand=True)

benchmark10 = pd.concat([benchmark9, fm_split], axis=1)
benchmark10.to_csv('benchmark10.csv', index=False)

In [ ]:
benchmark11 = pd.read_csv('benchmark11.csv')
split_string = '\|\|'
fm_split = benchmark11['QS_FastMoney_Q5'].str.split(split_string, 1, expand=True)
benchmark12 = pd.concat([benchmark11, fm_split], axis=1)
benchmark12.to_csv('benchmark12.csv', index=False)

In [ ]:
benchmark12 = pd.read_csv('benchmark12.csv')
split_string = ':'
game_header = benchmark11['GameTitle'].str.split(':', 1, expand=True)[0]
both_familes_title = benchmark11['GameTitle'].str.split(':', 1, expand=True)[1]

benchmark13 = pd.concat([benchmark12, game_header, both_familes_title], axis=1)
benchmark13.to_csv('benchmark13.csv', index=False)

In [ ]:
benchmark13 = pd.read_csv('benchmark13.csv')
split_string = ' vs | vs.| v | v.'
family_split = benchmark13['QS_FamiliesFull'].str.split(split_string, 1, expand=True)

benchmark14 = pd.concat([benchmark13, family_split], axis=1)
benchmark14.to_csv('benchmark14.csv', index=False)

In [ ]:
benchmark14 = pd.read_csv('benchmark14.csv')

split_string = '('
familyA_split = benchmark14['QS_FamilyAFull'].str.split(split_string, 1, expand=True)
familyB_split = benchmark14['QS_FamilyBFull'].str.split(split_string, 1, expand=True)

benchmark15 = pd.concat([benchmark14, familyA_split, familyB_split], axis=1)
benchmark15.to_csv('benchmark15.csv', index=False)

In [ ]:
split_string = '('
familyA_split = benchmark14['QS_FamilyAFull'].str.split(split_string, 1, expand=True)
familyB_split = benchmark14['QS_FamilyBFull'].str.split(split_string, 1, expand=True)

benchmark15 = pd.concat([benchmark14, familyA_split, familyB_split], axis=1)
benchmark15.to_csv('benchmark15.csv', index=False)

In [ ]:
benchmark15 = pd.read_csv('benchmark15.csv')

for col in benchmark15.columns:
    try:
        benchmark15[col] = benchmark15[col].str.strip()
    except:
        pass
    
benchmark15.to_csv('b16.csv', index=False)

In [ ]:
b16 = pd.read_csv('b16.csv')
b17 = b16
for letter in ['A','B']:
    fam_info_list = list(b16[f'QS_Family{letter}_Info'].str.split('\)|\('))
    fam_info_list = [fam_info_part if type(fam_info_part) == list else [] for fam_info_part in fam_info_list]
    
    def fam_info_sort(text):
        return -1*len(text.split(','))

    fam_info_list_sorted = [sorted(lis, key=fam_info_sort) for lis in fam_info_list]
    fam_info_list_sorted = [[li for li in lis if len(li)>0] for lis in fam_info_list_sorted]
    
    fam_info_df = pd.DataFrame(fam_info_list_sorted)
    
    b17 = pd.concat([b17, fam_info_df], axis=1)

b17.to_csv('b17.csv', index=False)

In [ ]:
b18 = pd.read_csv('b18.csv')

A = b18[['FamilyA','PostDate','GameID']]
A.columns = ['Family','PostDate','GameID']

B = b18[['FamilyB','PostDate','GameID']]
B.columns = ['Family','PostDate','GameID']

fam_dates = pd.concat([A, B], axis=0)
fam_dates['ID'] = np.nan
fam_dates['PostDate'] = pd.to_datetime(fam_dates['PostDate'], errors='coerce')

fam_dates = fam_dates.sort_values(by=['PostDate','GameID']).reset_index(drop=True)
fam_dates['PostDate'] = pd.to_datetime(fam_dates['PostDate'], errors='coerce')

b18['FamilyAID'] = np.nan
b18['FamilyBID'] = np.nan


window = 14 # days to consider the family the same
current_id = 1
for ind, row in fam_dates.iterrows():
    prev_games = fam_dates[(row['Family'] == fam_dates['Family']) & (fam_dates['PostDate'] >= row['PostDate'] - timedelta(days=window)) & (fam_dates['PostDate'] <= row['PostDate'])]
    if len(prev_games) == 1:
        fam_dates.loc[ind,'ID'] = current_id
        current_id+=1
    else:
        fam_dates.loc[ind,'ID'] = prev_games.iloc[0,2]

b18['PostDate'] = pd.to_datetime(b18['PostDate'], errors='coerce')
b19 = pd.merge(b18, fam_dates, left_on=['FamilyA', 'PostDate'], right_on=['Family','PostDate'])
b19 = pd.merge(b19, fam_dates, left_on=['FamilyB', 'PostDate'], right_on=['Family','PostDate'])

b19.to_csv('b19.csv', index=False)

In [ ]:
b20 = pd.read_csv('b20.csv')

A = b20[['FamilyA','PostDate','GameID']]
A.columns = ['Family','PostDate','GameID']

B = b20[['FamilyB','PostDate','GameID']]
B.columns = ['Family','PostDate','GameID']

fam_dates = pd.concat([A, B], axis=0)
fam_dates['ID'] = np.nan
fam_dates['PostDate'] = pd.to_datetime(fam_dates['PostDate'], errors='coerce')

fam_dates = fam_dates.sort_values(by=['PostDate','GameID']).reset_index(drop=True)
fam_dates['PostDate'] = pd.to_datetime(fam_dates['PostDate'], errors='coerce')

window = 14 # days to consider the family the same
current_id = 1
for ind, row in fam_dates.iterrows():
    prev_games = fam_dates[(row['Family'] == fam_dates['Family']) & (fam_dates['PostDate'] >= row['PostDate'] - timedelta(days=window)) & (fam_dates['PostDate'] <= row['PostDate'])]
    if len(prev_games) == 1:
        fam_dates.loc[ind,'ID'] = current_id
        current_id+=1
    else:
        fam_dates.loc[ind,'ID'] = prev_games.iloc[0,3]

b20['PostDate'] = pd.to_datetime(b20['PostDate'], errors='coerce')
b21 = pd.merge(b20, fam_dates, left_on=['FamilyA', 'PostDate'], right_on=['Family','PostDate'])
b21 = pd.merge(b21, fam_dates, left_on=['FamilyB', 'PostDate'], right_on=['Family','PostDate'])



b21.to_csv('b21.csv', index=False)

In [ ]:
b22 = pd.read_csv('b22.csv')

In [ ]:
a = b22[['FamilyA','FamilyAID','QS_FamilyA_Names','QS_FamilyA_Location','QS_FamilyA_Notes1','QS_FamilyA_Notes2']]
b = b22[['FamilyB','FamilyBID','QS_FamilyB_Names','QS_FamilyB_Location','QS_FamilyB_Notes1','QS_FamilyB_Notes2']]

In [ ]:
a = a.fillna("")
b = b.fillna("")

In [ ]:
fam_dict = {}
for ind, row in a.iterrows():
    if row['FamilyAID'] not in fam_dict.keys():
        fam_dict[row['FamilyAID']] = {}
    if 'Names' not in fam_dict[row['FamilyAID']].keys() or len(fam_dict[row['FamilyAID']]['Names']) < 5:
        fam_dict[row['FamilyAID']]['Names'] = row['QS_FamilyA_Names']
fam_dict_a = fam_dict

        
fam_dict = {}      
for ind, row in b.iterrows():
    if row['FamilyBID'] not in fam_dict.keys():
        fam_dict[row['FamilyBID']] = {}
    if 'Names' not in fam_dict[row['FamilyBID']].keys() or len(fam_dict[row['FamilyBID']]['Names']) < 5:
        fam_dict[row['FamilyBID']]['Names'] = row['QS_FamilyB_Names']
fam_dict_b = fam_dict

In [ ]:
# for key, value in sorted(fam_dict_b.items(), key=lambda x: x[0]): 
#     print("{} : {}".format(key, value))

In [ ]:
A = pd.DataFrame(fam_dict_a).T
B = pd.DataFrame(fam_dict_b).T
df = pd.merge(A, B, left_index=True, right_index=True, how='outer')

In [ ]:
df = df.fillna("")
df['longest'] = df.apply(lambda x: x['Names_x'] if len(x['Names_x']) > len(x['Names_y']) else x['Names_y'], axis=1)

In [ ]:
df = df[['longest']]

In [ ]:
df.to_csv('df.csv')

In [ ]:
b22 = pd.read_csv('b22.csv')

In [ ]:
name = "Abraham"
b23[((b23.FamilyA == name) | (b23.FamilyB == name))][['GameID','PostTitle','GameText','FamilyA','FamilyB','FamilyAID','FamilyBID']]


In [ ]:
id_ = 1936
b23[((b23.FamilyAID == id_) | (b23.FamilyBID == id_))][['GameID','PostTitle','GameText','FamilyA','FamilyB','FamilyAID','FamilyBID']]


In [142]:
b23 = pd.read_csv('b23.csv')
fams = pd.read_csv('fams.csv')

In [143]:
df = pd.merge(b23, fams, left_on='FamilyBID', right_on='FamilyID', how="left")

In [138]:
def name_in_game_text(row,n):
    return 1 if str(row[f'Name{n}']) in str(row['GameText']) else 0

for n in range(1,6):
    df[f'Name{n}InText'] = df.apply(lambda x: name_in_game_text(x,n), axis=1)
    
df['SumNameInText'] = df['Name1InText'] + df['Name2InText'] + df['Name3InText'] + df['Name4InText'] + df['Name5InText']

In [139]:
df['SumNameInText'].value_counts()

5    1226
4     497
3     304
2      95
1      12
0       2
Name: SumNameInText, dtype: int64

In [140]:
# df.iloc[6]['GameText']

In [141]:
name = "Simmons"
b23[((b23.FamilyA == name) | (b23.FamilyB == name))][['GameID','PostTitle','GameText','FamilyA','FamilyB','FamilyAID','FamilyBID']]


,GameID,PostTitle,GameText,FamilyA,FamilyB,FamilyAID,FamilyBID
431,432,“Family Feud” 11/4/13,"G1: Cohens v. Simmonses (LaTawnya, Katrina, De...",Cohen,Simmons,475.0,476.0
472,473,“Family Feud” 1/29/14,"G2: Tillmans v. Simmonses (Cole, Katie, Larry,...",Tillman,Simmons,521.0,524.0
474,475,“Family Feud” 1/30/14,"G2: Calhouns (Candy, Sandy, Kenny, John & Mich...",Calhoun,Simmons,526.0,524.0
476,477,“Family Feud” 1/31/14,"G2: Dickenses (Roland, Jimmy, Ricky, Rita & To...",Dickens,Simmons,528.0,524.0
478,479,“Family Feud” 2/3/14,G2: Goulds v. Simmonses || R1: Name an occupat...,Gould,Simmons,381.0,524.0
835,836,“Family Feud” 1/19,G1: Ramils v. Simmonses (they were here last s...,Ramil,Simmons,905.0,476.0
837,838,“Family Feud” 1/20/16,"G1: Cupps (Dayton)(Doug, Kelly, Mandy, Madison...",Cupp,Simmons,911.0,476.0
839,840,“Family Feud” 1/21,"M-U: Nathans (CHI)(Father Matt, Mark, Meredith...",Nathan,Simmons,914.0,476.0
840,841,“Family Feud” 1/25/16,"2ND CAR GAME THIS YR.: Millers (Jacksonville, ...",Miller,Simmons,915.0,476.0
1076,1077,“Family Feud” 2/16/17,"G1: Susinis v. Simmonses (DEN)(Sharon, Randell...",Susini,Simmons,1150.0,1157.0


In [134]:
df[(df['SumNameInText']==0)]

,GameID,PostTitle,GameText,GameHTML,PostVersion,PostDate,GameTitle,FamilyA_p,FamilyB_p,FamilyA,FamilyB,FamilyAID,FamilyBID,EpisodeNum,SeasonNum,QS_GameTitle,QS_Round1Tag,QS_Round1FullText,QS_Round2Tag,QS_Round2FullText,QS_DoubleTag,QS_DoubleFullText,QS_TripleTag,QS_TripleFullText,QS_SuddenDeathTag,QS_SuddenDeathFullText,QS_FastMoneyTag,QS_FastMoneyFullText,QS_Round1Q,QS_Round1A,QS_Round2Q,QS_Round2A,QS_DoubleQ,QS_DoubleA,QS_TripleQ,QS_TripleA,QS_SuddenDeathQ,QS_SuddenDeathA,0,QS_FastMoney_Q1_Tag,QS_FastMoney_Q1,QS_FastMoney_Q2_Tag,QS_FastMoney_Q2,QS_FastMoney_Q3_Tag,QS_FastMoney_Q3,QS_FastMoney_Q4_Tag,QS_FastMoney_Q4,QS_FastMoney_Q5_Tag,QS_FastMoney_Q5,QS_FastMoney_Answers,QS_GameHeader,QS_FamiliesFull,QS_FamilyAFull,QS_FamilyBFull,QS_FamilyA,QS_FamilyA_Info,QS_FamilyB,QS_FamilyB_Info,QS_FamilyA_Names,QS_FamilyA_Location,QS_FamilyA_Notes1,QS_FamilyA_Notes2,QS_FamilyB_Names,QS_FamilyB_Location,QS_FamilyB_Notes1,QS_FamilyB_Notes2,FamilyID,Names,Location,Notes,Name1,Name2,Name3,Name4,Name5,Name6,Name7,Name1InText,Name2InText,Name3InText,Name4InText,Name5InText,SumNameInText
68,69,“Family Feud” 11/3/10,|| R1 (women): Name something men wear that wo...,|| <p><strong>R1 (women): Name something men w...,Standard,2010-11-03,TNA Wrestling games (Wed),Wrestlers,Knockouts,Wrestler,Knockout,70.0,71.0,69,12,||,R1,(women): Name something men wear that women th...,R2,: Name another word for “stink”: || #1: Smell(...,|| Double,: Name a place where people AREN'T shy about m...,|| TRIPLE,: Name something an INCONSIDERATE person will ...,NaN,NaN,FM,:\n#1: Someone you'd be SHOCKED to receive a N...,(women): Name something men wear that women th...,#1:\n#2: Cologne/aftershave (12)(RVD)\n#3:\n#4...,: Name another word for “stink”: ||,#1: Smell(y)(72)(Velvet)\n#2:\n#3:\n#4:\n#5:\n...,: Name a place where people AREN'T shy about m...,#1: Park (38)(Angelina)\n#2:\n#3:\n#4:\n#5:\n#...,: Name something an INCONSIDERATE person will ...,#1: Food (44)(Lacey)\n#2:\n#3:\n#4: || Knockou...,NaN,NaN,:,#1,: Someone you'd be SHOCKED to receive a NAUGHT...,#2,: An insect you hope NEVER goes inside your ear.,#3,": Besides water, something used to put out fire.",#4,: Name the age when a girl should start wearin...,#5,: Something some men wear & some men DON'T.,"Rob: Mom (21), spider (21), CO² & 14 (18)\nKen...",TNA Wrestling games (Wed),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,"Angelina Love, Lacey Von Erich, Tara, Christy ...",NaN,NaN,Angelina Love,Lacey Von Erich,Tara,Christy Hemme,Velvet Sky,NaN,NaN,0,0,0,0,0,0
69,70,“Family Feud” 11/4/10,|| R1: Name a place a husband better NOT be wh...,|| <p><strong>R1: Name a place a husband bette...,Standard,2010-11-04,TNA Wrestling games (Thur),Wrestlers,Knockouts,Wrestler,Knockout,70.0,71.0,70,12,||,R1,: Name a place a husband better NOT be when hi...,R2,: Name a fruit a man thinks is sexy when a wom...,|| Double,: Name something some people do frequently & o...,|| TRIPLE,: Name something that's usually the job of the...,NaN,NaN,FM,:\n#1: Something your mom made you do before y...,: Name a place a husband better NOT be when hi...,#1: Bar (46)(Mick)\n#2: Girlfriend's house (19...,: Name a fruit a man thinks is sexy when a wom...,#1: Banana (67)(Jay)\n#2:\n#3:\n#4:\n#5:\n#6: ...,: Name something some people do frequently & o...,#1:\n#2:\n#3: Sex (6)(Christy)\n#4:\n#5:\n#6:\...,: Name something that's usually the job of the...,#1:\n#2:\n#3: Walk dog (5)(Velvet)\n#4:\nX- Ge...,NaN,NaN,:,#1,: Something your mom made you do before you le...,#2,": On the scale, how replaceable are you at work?",#3,: Something that might get broken in a bar fight.,#4,": Besides red, a color roses come in.",#5,: Which occupation would you find the bravest ...,"Matt: Make the bed (7), 0, bottle (31), white ...",TNA Wrestling games (Thur),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,"Angelina Love, Lacey Von Erich, Tara, Christy ...",NaN,NaN,Angelina Love,Lacey Von Erich,Tara,Christy Hemme,Velvet Sky,NaN,NaN

In [3]:
b24 = pd.read_csv('b24.csv')
fams = pd.read_csv('fams.csv')

In [19]:
As = b24[['FamilyAID','FamilyA','FamilyA_p']]
As.columns = ['FamilyID','Family','Family_p']
Bs = b24[['FamilyBID','FamilyB','FamilyB_p']]
Bs.columns = ['FamilyID','Family','Family_p']

In [20]:
df = pd.concat([As, Bs], axis=0)
df = df.drop_duplicates().sort_values(by='FamilyID')

In [21]:
fams_ = pd.merge(fams, df, left_on='FamilyID', right_on='FamilyID', how="left")

In [22]:
fams_.to_csv('fams_.csv', index=False)